In [1]:
import mpilot
import arcpy
import inspect
import pandas as pd
import geopandas as gpd
import os
import shutil
from shapely import wkt
from pyproj import CRS

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

# Summary

This notebook uses the mpilot library to build and run the EEMS model. The output is then joined to the original dataset using the `join_id` field using the Geopandas libary

**Input Data Layers:**
- Summarized Reporting Units (created in `4_Summarize.ipynb`)
    - Need both the csv and the shapefile


In [17]:
df = pd.read_csv("out_data/eems/A_RU_9.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18180 entries, 0 to 18179
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Join_ID     18180 non-null  int64  
 1   apn         18179 non-null  object 
 2   Pr_Lg_Fr    18180 non-null  float64
 3   Pr_Fire_Ig  18180 non-null  float64
 4   Mj_LU_Cat   18180 non-null  int64  
 5   WUI_Interm  18180 non-null  int64  
 6   WUI_Interf  18180 non-null  int64  
 7   sa_emb_rsk  18180 non-null  float64
 8   sd_emb_rsk  18180 non-null  float64
 9   bl_dns_0p5  18180 non-null  float64
 10  bl_dns_3mi  18180 non-null  float64
 11  Shape_Leng  18180 non-null  float64
 12  Shape_Area  18180 non-null  float64
 13  bl_dns_1kf  18180 non-null  float64
 14  geometry    18180 non-null  object 
 15  LU_Cat      18180 non-null  float64
dtypes: float64(10), int64(4), object(2)
memory usage: 2.2+ MB


**Create the mpilot command file**

In [18]:
def get_base_str(reporting_units_path, output_path):
    base_str = f"""

# Read Commands
Join_ID = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Join_ID",
    DataType = "Integer",
    Metadata = [
        DisplayName: Join&nbsp;ID,
        Description: Join&nbsp;ID,
        ColorMap: RdYlBu_r
    ]
)

Mj_LU_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Mj_LU_Cat",
    DataType = "Integer",
    Metadata = [
        DisplayName: Majority&nbsp;LandUse&nbsp;Category,
        Description: Majority&nbsp;LandUse&nbsp;Category,
        ColorMap: RdYlBu_r
    ]
)

Pr_Lg_Fr = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Lg_Fr",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        Description: Relative&nbsp;Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Fire_Ig = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Fire_Ig",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interm = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interm",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Intermix,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interf = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interf",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Interface,
        ColorMap: RdYlBu_r
    ]
)

sa_emb_rsk = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "sa_emb_rsk",
    DataType = "Float",
    Metadata = [
        DisplayName: Santa&nbsp;Ana&nbsp;Ember&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

sd_emb_rsk = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "sd_emb_rsk",
    DataType = "Float",
    Metadata = [
        DisplayName: Sundowner&nbsp;Ember&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

bl_dns_0p5 = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "bl_dns_0p5",
    DataType = "Integer",
    Metadata = [
        DisplayName: Half&nbsp;Mile&nbsp;Building&nbsp;Density&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

bl_dns_3mi = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "bl_dns_3mi",
    DataType = "Integer",
    Metadata = [
        DisplayName: Three&nbsp;Mile&nbsp;Building&nbsp;Density&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

bl_dns_1kf = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "bl_dns_1kf",
    DataType = "Integer",
    Metadata = [
        DisplayName: Thousand&nbsp;Feet&nbsp;Building&nbsp;Density&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Fz = CvtToFuzzyCat(
    InFieldName = Mj_LU_Cat, 
    RawValues = [0,1,2,3,4], 
    FuzzyValues = [1,0,0.1,0,0], 
    DefaultFuzzyValue = 0,
    Metadata = [
        DisplayName: Fuzzy&nbsp;LandUse&nbsp;Category,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Cvt = CvtFromFuzzy(
    InFieldName = LU_Cat_Fz, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Converted&nbsp;LandUse&nbsp;Category,
        ColorMap: RdYlBu_r
    ]
)



# Normalizations

Nm_Lg_Fire = Normalize(
    InFieldName = Pr_Lg_Fr, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        ColorMap: RdYlBu_r
    ]
)

Nm_Fire_Ig = Normalize(
    InFieldName = Pr_Fire_Ig, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition,
        ColorMap: RdYlBu_r
    ]
)

nm_dns_1kf = Normalize(
    InFieldName = bl_dns_1kf, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Thousand&nbsp;Feet&nbsp;Building&nbsp;Density,
        ColorMap: RdYlBu_r
    ]
)

nm_dns_0p5 = Normalize(
    InFieldName = bl_dns_0p5, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Half&nbsp;Mile&nbsp;Building&nbsp;Density,
        ColorMap: RdYlBu_r
    ]
)

nm_dns_3mi = Normalize(
    InFieldName = bl_dns_3mi, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Three&nbsp;Mile&nbsp;Building&nbsp;Density,
        ColorMap: RdYlBu_r
    ]
)

nm_sa_rsk = Normalize(
    InFieldName = sa_emb_rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Santa&nbsp;Ana&nbsp;Ember&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

nm_sd_rsk = Normalize(
    InFieldName = sd_emb_rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Sundowner&nbsp;Ember&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

# Operations

embr_rsk = WeightedSum(
    InFieldNames = [nm_sd_rsk, nm_sa_rsk], 
    Weights = [0.5, 0.5],
    Metadata = [
        DisplayName: Combined&nbsp;Ember&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

nm_emb_rsk = Normalize(
    InFieldName = embr_rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Combined&nbsp;Ember&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

dens_dev = WeightedSum(
    InFieldNames = [nm_dns_1kf, nm_dns_0p5, nm_dns_3mi], 
    Weights = [0.33, 0.33, 0.34],
    Metadata = [
        DisplayName: Proximity&nbsp;To&nbsp;Dense&nbsp;Development&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

nm_den_dev = Normalize(
    InFieldName = dens_dev, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName:  Proximity&nbsp;To&nbsp;Dense&nbsp;Development,
        ColorMap: RdYlBu_r
    ]
)

WUI_Rsk = Maximum(
    InFieldNames = [WUI_Interm, WUI_Interf],
    Metadata = [
        DisplayName: WUI&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

Hi_Pri = WeightedSum(
    InFieldNames = [Nm_Lg_Fire, Nm_Fire_Ig, nm_emb_rsk, nm_den_dev, WUI_Rsk, LU_Cat_Cvt], 
    Weights = [0.16, 0.2, 0.16, 0.16, 0.16, 0.16],
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Area&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Nm_Hi_Pri = Normalize(
    InFieldName = Hi_Pri, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Area,
        ColorMap: RdYlBu_r
    ]
)

# Output

Out = EEMSWrite(
  OutFileName = {output_path},
  OutFieldNames = [Join_ID, Nm_Hi_Pri, Hi_Pri, WUI_Rsk, Nm_Lg_Fire, Nm_Fire_Ig, WUI_Interm, WUI_Interf, Pr_Lg_Fr, Pr_Fire_Ig, LU_Cat_Cvt, LU_Cat_Fz, Mj_LU_Cat,
                   nm_den_dev, dens_dev, nm_emb_rsk, embr_rsk, nm_sd_rsk, nm_sa_rsk, nm_dns_3mi, nm_dns_0p5, nm_dns_1kf,
                   bl_dns_1kf, bl_dns_3mi, bl_dns_0p5, sd_emb_rsk, sa_emb_rsk]
)
    """
    return base_str

In [19]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/A_RU_9.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output.csv")
base_str = get_base_str(reporting_units_path, output_path)
f = open("EEMS_CMD_FILE.mpt", "w")
f.write(base_str)
f.close()

**Run EEMS Command `mpilot eems-csv [command_file]`**

**Now we need to merge the outputted EEMS CSV with the rest of the data and create a shapefile for it. Also note we need to project it to UTM Zone 11N so that it visualizes in EEMS Online**

In [20]:
dir_path = os.path.join(THIS_FOLDER, "out_data/final")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

eems = pd.merge(pd.read_csv("out_data/eems/EEMS_Output.csv"), pd.read_csv("out_data/eems/A_RU_9.csv")[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
crs_proj = CRS("EPSG:2229")
gdf = gpd.GeoDataFrame(eems, crs=crs_proj, geometry = eems.geometry)
gdf.to_file(driver = 'ESRI Shapefile', filename= "out_data/final/Final_EEMS_Output.shp")

**Now prep the Command File and the geodatabase for EEMS Online**

In [21]:
arcpy.ResetEnvironments()
gdb_name = "EEMS.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path
path = os.path.join(THIS_FOLDER, "out_data/final/Final_EEMS_Output.shp")
FeatureClass = arcpy.CopyFeatures_management(path, "EEMS")
crs = arcpy.SpatialReference('NAD 1983 UTM Zone 11N')
FeatureClass = arcpy.Project_management("EEMS", "EEMS_rpj", crs)
arcpy.management.Delete("EEMS")

<Result 'true'>

In [22]:
f1 = open("EEMS_CMD_FILE.mpt", 'r')
f2 = open("EEMS_CMD_FILE_2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

In [23]:
arcpy.ClearWorkspaceCache_management()
del FeatureClass

**Now delete the EEMSWRITE command from the Command File and zip the geodatabase with the final EEMS feature class (and only that feature class)**

In [24]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 18180 entries, 0 to 18179
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Join_ID     18180 non-null  float64 
 1   Nm_Hi_Pri   18180 non-null  float64 
 2   Hi_Pri      18180 non-null  float64 
 3   WUI_Rsk     18180 non-null  float64 
 4   Nm_Lg_Fire  18180 non-null  float64 
 5   Nm_Fire_Ig  18180 non-null  float64 
 6   WUI_Interm  18180 non-null  float64 
 7   WUI_Interf  18180 non-null  float64 
 8   Pr_Lg_Fr    18180 non-null  float64 
 9   Pr_Fire_Ig  18180 non-null  float64 
 10  LU_Cat_Cvt  18180 non-null  float64 
 11  LU_Cat_Fz   18180 non-null  float64 
 12  Mj_LU_Cat   18180 non-null  float64 
 13  nm_den_dev  18180 non-null  float64 
 14  dens_dev    18180 non-null  float64 
 15  nm_emb_rsk  18180 non-null  float64 
 16  embr_rsk    18180 non-null  float64 
 17  nm_sd_rsk   18180 non-null  float64 
 18  nm_sa_rsk   18180 non-null  float64 
 

In [25]:
gdf['sa_emb_rsk']

array([0.729508  , 0.80401802, 0.80191101, ..., 0.893795  , 0.353488  ,
       1.246094  ])